In [3]:
import cv2
import random
import os

def find_video_by_frame_index(video_frame_ranges, frame_index):
    for video_file, (start, end) in video_frame_ranges.items():
        if start <= frame_index <= end:
            return video_file, frame_index - start
    return None, None

def sample_n_frames_uniformly_from_folder(folder_path, n_samples):
    video_files = [f for f in os.listdir(folder_path) if f.endswith('.avi')]
    total_frames = 0
    video_frame_ranges = {}

    # Calculate the cumulative frame count and store frame ranges for each video
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_file}.")
            continue
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_frame_ranges[video_file] = (total_frames, total_frames + frame_count - 1)
        total_frames += frame_count
        cap.release()

    # Adjust n_samples if it exceeds the total frame count
    n_samples = min(n_samples, total_frames)

    # Randomly select n frame indices from the total frame pool
    selected_frame_indices = random.sample(range(total_frames), n_samples)

    # Map selected frame indices to their corresponding videos and frame positions
    for frame_index in selected_frame_indices:
        video_file, local_frame_index = find_video_by_frame_index(video_frame_ranges, frame_index)
        if video_file is not None:
            video_path = os.path.join(folder_path, video_file)
            cap = cv2.VideoCapture(video_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, local_frame_index)
            ret, frame = cap.read()
            frame_file_name = f"{os.path.splitext(video_file)[0]}_frame_{local_frame_index}.png"
            cv2.imwrite(frame_file_name, frame)
            cap.release()

    


# Usage
video_path = './test_data/'  # Replace with your video path
n_samples = 80  # Number of frames you want to sample

sampled_frames = sample_n_frames_uniformly_from_folder(video_path, n_samples)